In [1]:
import os
import sys
sys.path.append('../')

import src.module_process_energies as proc_ener
import src.module_miscellaneous as misc
import src.module_mainfunctions as mfunc

In [2]:
convRyd2eV = 13.6057 # eV/Ryd
convcm2Ryd = 1./109737.26 
temperature = misc.default_temperature_grid()

# Autostructure data input parameters:
file_om = "adf04ls"
file_ups = "adf04ls_ups"
nterms = 189 # hacer una función para automatizar esto (leyendo previamente adf04)
ntran = 12389 # hacer una función para automatizar esto (leyendo previamente adf04)
folder = "../de_ishftls/"
AScalc = 'AS(ishftls)'


# Read database (DB) values
nsuplev = 55
dbfilepath = '~/magnesio/resumen/ModelAtomicIonLevel-IMPORTED.txt'
terms_DB = proc_ener.read_database_energies(dbfilepath)
compterms = {
    'database(NIST)': {
        'terms': terms_DB, 
        'suplev': nsuplev
    }
} # => index where superlevels appear

In [3]:
# read Autostructure calculation and database data for energy values
terms_AS, termdict_DB = mfunc.energy_data(folder, file_om, AScalc, nterms, compterms)

checking energy conversion:
3S1 3P1 21890.8529 cm^2 =>  0.19948423078906838 Ryd

=> warning: using superlevels predefined
... all done!



In [4]:
# read CS and ECS (compute them if they have not been computed thus far) 
# => this is accurate only when shifted energies have been included in the EIE calculation
omg, ups = mfunc.collisional_data(folder, file_om, terms_AS, nterms, ntran)

sorting transition type...
... all done!
 ==> 4917 transition were dropped because eik==0



In [5]:
# rename effective collision strengths using database index
ups_r = mfunc.rename_transition_levels(ups, termdict_DB)

processing index transformation..
... all done!


In [6]:
# compute superlevel with formula 1
ups_slv = mfunc.compute_superlevels_ECS(terms_AS, ups_r, nlevmax=85, nlevmin=84)

In [7]:
ups_slv.head()

,k,i,3000.0,4000.0,5000.0,6000.0,7000.0,8000.0,9000.0,10000.0,11000.0,12000.0,13000.0,14000.0,15000.0,16000.0
0,2,1,2.166095,2.675268,3.100920,3.459414,3.763363,4.022538,4.244591,4.435577,4.600340,4.742796,4.866141,4.973006,5.065580,5.145693
1,3,1,2.859619,3.148998,3.393255,3.612149,3.815352,4.008193,4.193846,4.374315,4.550918,4.724561,4.895880,5.065338,5.233275,5.399946
2,3,2,12.722873,13.983362,14.822721,15.385166,15.756620,15.992047,16.128338,16.191139,16.198774,16.164621,16.098627,16.008295,15.899357,15.776239
3,4,1,0.973188,1.060260,1.123046,1.169354,1.203607,1.228669,1.246548,1.258730,1.266349,1.270289,1.271252,1.269797,1.266379,1.261366
4,4,2,7.436320,8.151954,8.754221,9.291934,9.788039,10.254843,10.699624,11.127033,11.540245,11.941546,12.332659,12.714926,13.089422,13.457021


In [8]:
filepath = os.path.join(folder, 'ECS-AS-superlevels.dat')
ups_slv.to_csv(filepath, index=False)

In [9]:
len(ups_slv)

3557

In [10]:
len(ups)

7472